### Support Vector Regression

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append("..")
from model_validation import ModelValidation

ImportError: attempted relative import with no known parent package